In [1]:
library(tidyverse)
library(data.table)
#turning off warnings
options(warn=-1)
options(scipen = 999)
#setting plot width and height
options(repr.plot.width=15, repr.plot.height = 10)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




In [2]:
comp_prob <- fread('../input/merge-clean-data/full_tracking_df.csv',
                  drop='V1')
coverage <- fread('../input/coverage-id/extra_data.csv',
                 drop='V1')
colnames(comp_prob)
colnames(coverage)
# things to layer in:
# observed completion %, scale by time required to cover (snap_to_throw), 
# cluster based on output metrics AND style

[1] "gameId"                    "playId"                   
 [3] "frameId"                   "off_nflId"                
 [5] "nflId"                     "displayName"              
 [7] "dist_from_off"             "nearest_is_target"        
 [9] "distanceFromBall.x"        "off_distanceFromBall"     
[11] "ball_behind_db"            "diff_in_speed"            
[13] "diff_in_acceleration"      "diff_in_dir"              
[15] "receiver_behind_db"        "receiver_distance_from_qb"
[17] "num_of_receivers_nearest"  "dist_rate_of_change"      
[19] "his_off_most_open"         "passResult"               
[21] "targetNflId"               "x"                        
[23] "y"                         "o"                        
[25] "dir"                       "s"                        
[27] "a"                         "event"                    
[29] "epa"                       "position"                 
[31] "route"                     "isDefensivePI"            
[33] "targetedReceiver"          "team_name"                
[35] "ball_x"                    "ball_y"                   
[37] "ball_snap_x"               "max_space_available"      
[39] "target_x"                  "target_y"                 
[41] "qb_x"                      "qb_y"                     
[43] "qb_s"                      "qb_a"                     
[45] "distanceFromBall.y"        "distanceFromTarget"       
[47] "distance_from_qb"

[1] "gameId"         "playId"         "nflId"          "snap_to_throw" 
 [5] "var_xs"         "var_ys"         "var_o"          "avg_speed"     
 [9] "var_speed"      "look_at_qb"     "avg_dist"       "var_dist"      
[13] "avg_dir_diff"   "avg_speed_diff" "pct_first_off"  "pct_last_off"  
[17] "last_off"       "first_off"      "var_look_at_qb" "dist_los"

In [3]:
head(comp_prob)
head(coverage)

gameId,playId,frameId,off_nflId,nflId,displayName,dist_from_off,nearest_is_target,distanceFromBall.x,off_distanceFromBall,⋯,max_space_available,target_x,target_y,qb_x,qb_y,qb_s,qb_a,distanceFromBall.y,distanceFromTarget,distance_from_qb
<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018090600,75,11,2533040,79848,Malcolm Jenkins,1.920339,0,9.726813,9.334077,⋯,90.11,28.64,9.193333,28.26,26.66333,0.00,0.00,9.726813,8.029352,10.23481
2018090600,75,12,2533040,79848,Malcolm Jenkins,1.922134,0,9.736735,9.334077,⋯,90.11,28.65,9.193333,28.24,26.65333,0.00,0.00,9.736735,8.016764,10.24040
2018090600,75,13,2533040,79848,Malcolm Jenkins,1.912302,0,9.762797,9.365452,⋯,90.11,28.68,9.193333,28.22,26.65333,0.03,0.82,9.762797,8.007609,10.24602
2018090600,75,14,2533040,79848,Malcolm Jenkins,1.908743,0,9.814622,9.353614,⋯,90.11,28.72,9.193333,28.21,26.65333,0.22,2.24,9.814622,8.005105,10.23926
2018090600,75,15,2533040,79848,Malcolm Jenkins,1.889047,0,9.884452,9.351048,⋯,90.11,28.76,9.203333,28.16,26.65333,0.61,3.46,9.884452,7.977092,10.26599
2018090600,75,16,2533040,79848,Malcolm Jenkins,1.859516,0,9.927724,9.341333,⋯,90.11,28.84,9.213333,28.06,26.66333,1.18,4.58,9.927724,7.937512,10.31733


gameId,playId,nflId,snap_to_throw,var_xs,var_ys,var_o,avg_speed,var_speed,look_at_qb,avg_dist,var_dist,avg_dir_diff,avg_speed_diff,pct_first_off,pct_last_off,last_off,first_off,var_look_at_qb,dist_los
<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>
2018090600,75,79848,2.5,1.70033046,0.22510538,5583.62867,1.871154,3.419619,0.6923077,2.038183,0.7144427,59.86346,-1.4450000,1.0000000,1.00000000,2533040,2533040,0.2215385,1.22
2018090600,75,2495613,2.5,0.10720446,1.20087646,333.14461,1.601923,1.279824,1.0000000,3.121003,1.8412830,96.86885,-1.6265385,0.9615385,0.03846154,2559033,2555415,0.0000000,3.64
2018090600,75,2534832,2.5,1.84011154,0.43926462,128.65881,1.780000,1.614448,1.0000000,17.105852,2.7880629,38.29346,-1.5361538,1.0000000,1.00000000,2533040,2533040,0.0000000,16.50
2018090600,75,2552689,2.5,0.97037538,0.02726846,13.05627,1.246154,1.052137,1.0000000,7.093199,2.6571374,14.50462,-1.9615385,1.0000000,1.00000000,2555415,2555415,0.0000000,7.48
2018090600,75,2555383,2.5,6.13310046,0.27971954,1496.27083,2.948077,4.262440,1.0000000,5.678933,2.0882765,27.34846,-1.1592308,1.0000000,1.00000000,2495454,2495454,0.0000000,6.19
2018090600,146,79848,2.0,0.01500143,2.61227476,139.61959,2.519048,2.972869,0.6666667,2.538371,1.1541539,150.75381,-0.4657143,0.8095238,0.19047619,2555415,2560854,0.2333333,2.62


In [4]:
full_data <- left_join(comp_prob,
                      coverage,
                      by=c('gameId', 'playId', 'nflId'))
nrow(full_data)
head(full_data)
colnames(full_data)

[1] 3525934

gameId,playId,frameId,off_nflId,nflId,displayName,dist_from_off,nearest_is_target,distanceFromBall.x,off_distanceFromBall,⋯,avg_dist,var_dist,avg_dir_diff,avg_speed_diff,pct_first_off,pct_last_off,last_off,first_off,var_look_at_qb,dist_los
<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>
2018090600,75,11,2533040,79848,Malcolm Jenkins,1.920339,0,9.726813,9.334077,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22
2018090600,75,12,2533040,79848,Malcolm Jenkins,1.922134,0,9.736735,9.334077,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22
2018090600,75,13,2533040,79848,Malcolm Jenkins,1.912302,0,9.762797,9.365452,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22
2018090600,75,14,2533040,79848,Malcolm Jenkins,1.908743,0,9.814622,9.353614,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22
2018090600,75,15,2533040,79848,Malcolm Jenkins,1.889047,0,9.884452,9.351048,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22
2018090600,75,16,2533040,79848,Malcolm Jenkins,1.859516,0,9.927724,9.341333,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22


[1] "gameId"                    "playId"                   
 [3] "frameId"                   "off_nflId"                
 [5] "nflId"                     "displayName"              
 [7] "dist_from_off"             "nearest_is_target"        
 [9] "distanceFromBall.x"        "off_distanceFromBall"     
[11] "ball_behind_db"            "diff_in_speed"            
[13] "diff_in_acceleration"      "diff_in_dir"              
[15] "receiver_behind_db"        "receiver_distance_from_qb"
[17] "num_of_receivers_nearest"  "dist_rate_of_change"      
[19] "his_off_most_open"         "passResult"               
[21] "targetNflId"               "x"                        
[23] "y"                         "o"                        
[25] "dir"                       "s"                        
[27] "a"                         "event"                    
[29] "epa"                       "position"                 
[31] "route"                     "isDefensivePI"            
[33] "targetedReceiver"          "team_name"                
[35] "ball_x"                    "ball_y"                   
[37] "ball_snap_x"               "max_space_available"      
[39] "target_x"                  "target_y"                 
[41] "qb_x"                      "qb_y"                     
[43] "qb_s"                      "qb_a"                     
[45] "distanceFromBall.y"        "distanceFromTarget"       
[47] "distance_from_qb"          "snap_to_throw"            
[49] "var_xs"                    "var_ys"                   
[51] "var_o"                     "avg_speed"                
[53] "var_speed"                 "look_at_qb"               
[55] "avg_dist"                  "var_dist"                 
[57] "avg_dir_diff"              "avg_speed_diff"           
[59] "pct_first_off"             "pct_last_off"             
[61] "last_off"                  "first_off"                
[63] "var_look_at_qb"            "dist_los"

In [5]:
head(full_data)
colnames(full_data)

gameId,playId,frameId,off_nflId,nflId,displayName,dist_from_off,nearest_is_target,distanceFromBall.x,off_distanceFromBall,⋯,avg_dist,var_dist,avg_dir_diff,avg_speed_diff,pct_first_off,pct_last_off,last_off,first_off,var_look_at_qb,dist_los
<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>
2018090600,75,11,2533040,79848,Malcolm Jenkins,1.920339,0,9.726813,9.334077,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22
2018090600,75,12,2533040,79848,Malcolm Jenkins,1.922134,0,9.736735,9.334077,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22
2018090600,75,13,2533040,79848,Malcolm Jenkins,1.912302,0,9.762797,9.365452,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22
2018090600,75,14,2533040,79848,Malcolm Jenkins,1.908743,0,9.814622,9.353614,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22
2018090600,75,15,2533040,79848,Malcolm Jenkins,1.889047,0,9.884452,9.351048,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22
2018090600,75,16,2533040,79848,Malcolm Jenkins,1.859516,0,9.927724,9.341333,⋯,2.038183,0.7144427,59.86346,-1.445,1,1,2533040,2533040,0.2215385,1.22


[1] "gameId"                    "playId"                   
 [3] "frameId"                   "off_nflId"                
 [5] "nflId"                     "displayName"              
 [7] "dist_from_off"             "nearest_is_target"        
 [9] "distanceFromBall.x"        "off_distanceFromBall"     
[11] "ball_behind_db"            "diff_in_speed"            
[13] "diff_in_acceleration"      "diff_in_dir"              
[15] "receiver_behind_db"        "receiver_distance_from_qb"
[17] "num_of_receivers_nearest"  "dist_rate_of_change"      
[19] "his_off_most_open"         "passResult"               
[21] "targetNflId"               "x"                        
[23] "y"                         "o"                        
[25] "dir"                       "s"                        
[27] "a"                         "event"                    
[29] "epa"                       "position"                 
[31] "route"                     "isDefensivePI"            
[33] "targetedReceiver"          "team_name"                
[35] "ball_x"                    "ball_y"                   
[37] "ball_snap_x"               "max_space_available"      
[39] "target_x"                  "target_y"                 
[41] "qb_x"                      "qb_y"                     
[43] "qb_s"                      "qb_a"                     
[45] "distanceFromBall.y"        "distanceFromTarget"       
[47] "distance_from_qb"          "snap_to_throw"            
[49] "var_xs"                    "var_ys"                   
[51] "var_o"                     "avg_speed"                
[53] "var_speed"                 "look_at_qb"               
[55] "avg_dist"                  "var_dist"                 
[57] "avg_dir_diff"              "avg_speed_diff"           
[59] "pct_first_off"             "pct_last_off"             
[61] "last_off"                  "first_off"                
[63] "var_look_at_qb"            "dist_los"

In [6]:
full_data <- full_data %>%

mutate(complete = if_else(passResult == 'C', 1, 0),
      covering_target = if_else(last_off == targetNflId & pct_last_off >= .33, 1, 0),
      covering_first_off = if_else(pct_first_off >= .25 & first_off == off_nflId, 1, 0))
head(full_data)

gameId,playId,frameId,off_nflId,nflId,displayName,dist_from_off,nearest_is_target,distanceFromBall.x,off_distanceFromBall,⋯,avg_speed_diff,pct_first_off,pct_last_off,last_off,first_off,var_look_at_qb,dist_los,complete,covering_target,covering_first_off
<int>,<int>,<int>,<int>,<int>,<chr>,<dbl>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018090600,75,11,2533040,79848,Malcolm Jenkins,1.920339,0,9.726813,9.334077,⋯,-1.445,1,1,2533040,2533040,0.2215385,1.22,1,0,1
2018090600,75,12,2533040,79848,Malcolm Jenkins,1.922134,0,9.736735,9.334077,⋯,-1.445,1,1,2533040,2533040,0.2215385,1.22,1,0,1
2018090600,75,13,2533040,79848,Malcolm Jenkins,1.912302,0,9.762797,9.365452,⋯,-1.445,1,1,2533040,2533040,0.2215385,1.22,1,0,1
2018090600,75,14,2533040,79848,Malcolm Jenkins,1.908743,0,9.814622,9.353614,⋯,-1.445,1,1,2533040,2533040,0.2215385,1.22,1,0,1
2018090600,75,15,2533040,79848,Malcolm Jenkins,1.889047,0,9.884452,9.351048,⋯,-1.445,1,1,2533040,2533040,0.2215385,1.22,1,0,1
2018090600,75,16,2533040,79848,Malcolm Jenkins,1.859516,0,9.927724,9.341333,⋯,-1.445,1,1,2533040,2533040,0.2215385,1.22,1,0,1


In [7]:
colnames(full_data)

# kpis: stickiness, speed, contested catch scenarios, break on ball
#(scaled against overall xComp, just simple change in dist_rate_of_change???),
# recovery speed (when x dist behind who gains), height, 

# instead of identifying coverage, just ask, was he covering this guy?
# take two or three key indicators of what you're looking at then regress them

[1] "gameId"                    "playId"                   
 [3] "frameId"                   "off_nflId"                
 [5] "nflId"                     "displayName"              
 [7] "dist_from_off"             "nearest_is_target"        
 [9] "distanceFromBall.x"        "off_distanceFromBall"     
[11] "ball_behind_db"            "diff_in_speed"            
[13] "diff_in_acceleration"      "diff_in_dir"              
[15] "receiver_behind_db"        "receiver_distance_from_qb"
[17] "num_of_receivers_nearest"  "dist_rate_of_change"      
[19] "his_off_most_open"         "passResult"               
[21] "targetNflId"               "x"                        
[23] "y"                         "o"                        
[25] "dir"                       "s"                        
[27] "a"                         "event"                    
[29] "epa"                       "position"                 
[31] "route"                     "isDefensivePI"            
[33] "targetedReceiver"          "team_name"                
[35] "ball_x"                    "ball_y"                   
[37] "ball_snap_x"               "max_space_available"      
[39] "target_x"                  "target_y"                 
[41] "qb_x"                      "qb_y"                     
[43] "qb_s"                      "qb_a"                     
[45] "distanceFromBall.y"        "distanceFromTarget"       
[47] "distance_from_qb"          "snap_to_throw"            
[49] "var_xs"                    "var_ys"                   
[51] "var_o"                     "avg_speed"                
[53] "var_speed"                 "look_at_qb"               
[55] "avg_dist"                  "var_dist"                 
[57] "avg_dir_diff"              "avg_speed_diff"           
[59] "pct_first_off"             "pct_last_off"             
[61] "last_off"                  "first_off"                
[63] "var_look_at_qb"            "dist_los"                 
[65] "complete"                  "covering_target"          
[67] "covering_first_off"

In [8]:
summary <- full_data %>%

filter(covering_first_off == 1) %>%

group_by(nflId, displayName) %>%

summarise(avg_xComp = mean(comp_prob),
         frames = n(),
         avg_dist = mean(dist_from_off),
         var_dist = mean(var_dist),
         dist_los = mean(dist_los)) %>%

filter(frames > 1500) %>%

arrange(avg_xComp)
head(summary, 10)

`summarise()` regrouping output by 'nflId' (override with `.groups` argument)



nflId,displayName,avg_xComp,frames,avg_dist,var_dist,dist_los
<int>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
306,Dominique Rodgers-Cromartie,NA,2373,4.493909,3.394613,5.995065
1302,Aqib Talib,NA,6039,5.887950,2.953327,6.991921
2307,Orlando Scandrick,NA,15805,3.853977,2.233640,NA
4365,Brandon Carr,NA,17727,4.013756,2.190479,4.448796
71251,Patrick Chung,NA,15836,4.459495,4.744256,4.514494
71441,Glover Quin,NA,11176,9.225391,5.637782,10.877035
79848,Malcolm Jenkins,NA,16104,5.707513,3.687130,6.205839
80668,Captain Munnerlyn,NA,11359,3.729332,2.366336,3.992982
89756,Jason McCourty,NA,16998,3.964662,4.450241,5.126608


In [9]:
fixHeight <- function(str){
    str <- str_split(str, '-', simplify=FALSE)
    str <- (as.numeric(str[[1]][1]) * 12) + as.numeric(str[[1]][2])
    return(str)
}
fixHeight('6-0')

[1] 72

In [10]:
filter_nearests <- full_data %>%

filter(event == 'pass_forward' & covering_target == 1) %>%

select(dist_from_off, playId, gameId, nflId) %>%

group_split(gameId, playId)

In [11]:
get_closest <- function(play_df){
    min_dist <- min(play_df$dist_from_off)
    min_player <- which(play_df$dist_from_off == min_dist)
    
    return(play_df[min_player,])
}
get_closest(filter_nearests[[1]])

dist_from_off,playId,gameId,nflId
<dbl>,<int>,<int>,<int>
4.160048,75,2018090600,2555383


In [12]:
closest <- bind_rows(lapply(filter_nearests, get_closest))
head(closest)

dist_from_off,playId,gameId,nflId
<dbl>,<int>,<int>,<int>
4.160048,75,2018090600,2555383
4.687569,146,2018090600,2555383
4.243218,190,2018090600,79848
3.989010,320,2018090600,2539653
2.144108,344,2018090600,2543850
3.560351,402,2018090600,2552689


In [13]:
rand_forest <- left_join(closest,
                        full_data,
                        by=c('gameId', 'playId', 'dist_from_off', 'nflId'))
head(rand_forest)

dist_from_off,playId,gameId,nflId,frameId,off_nflId,displayName,nearest_is_target,distanceFromBall.x,off_distanceFromBall,⋯,avg_speed_diff,pct_first_off,pct_last_off,last_off,first_off,var_look_at_qb,dist_los,complete,covering_target,covering_first_off
<dbl>,<int>,<int>,<int>,<int>,<int>,<chr>,<int>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4.160048,75,2018090600,2555383,36,2495454,Jalen Mills,1,27.15692,24.17866,⋯,-1.159231,1.0000000,1.0000000,2495454,2495454,0.0000000,6.19,1,1,1
4.687569,146,2018090600,2555383,31,2560854,Jalen Mills,1,16.22502,11.62252,⋯,-1.004286,1.0000000,1.0000000,2560854,2560854,0.0000000,7.56,0,1,1
4.243218,190,2018090600,79848,39,2495454,Malcolm Jenkins,1,19.23593,23.47701,⋯,-2.902759,1.0000000,1.0000000,2495454,2495454,0.0000000,4.31,1,1,1
3.989010,320,2018090600,2539653,29,2552600,Robert Alford,1,17.72055,14.54602,⋯,-1.560526,0.1578947,0.8421053,2552600,2540158,0.2280702,3.89,1,1,0
2.144108,344,2018090600,2543850,34,2506467,Ricardo Allen,1,17.08980,15.13528,⋯,-2.086667,0.4583333,0.5416667,2506467,2507763,0.1141304,6.76,0,1,0
3.560351,402,2018090600,2552689,46,2533040,Ronald Darby,1,32.91137,29.60650,⋯,-1.440556,1.0000000,1.0000000,2533040,2533040,0.0000000,8.16,0,1,1


In [14]:
nrow(rand_forest)
colnames(rand_forest)

[1] 12645

[1] "dist_from_off"             "playId"                   
 [3] "gameId"                    "nflId"                    
 [5] "frameId"                   "off_nflId"                
 [7] "displayName"               "nearest_is_target"        
 [9] "distanceFromBall.x"        "off_distanceFromBall"     
[11] "ball_behind_db"            "diff_in_speed"            
[13] "diff_in_acceleration"      "diff_in_dir"              
[15] "receiver_behind_db"        "receiver_distance_from_qb"
[17] "num_of_receivers_nearest"  "dist_rate_of_change"      
[19] "his_off_most_open"         "passResult"               
[21] "targetNflId"               "x"                        
[23] "y"                         "o"                        
[25] "dir"                       "s"                        
[27] "a"                         "event"                    
[29] "epa"                       "position"                 
[31] "route"                     "isDefensivePI"            
[33] "targetedReceiver"          "team_name"                
[35] "ball_x"                    "ball_y"                   
[37] "ball_snap_x"               "max_space_available"      
[39] "target_x"                  "target_y"                 
[41] "qb_x"                      "qb_y"                     
[43] "qb_s"                      "qb_a"                     
[45] "distanceFromBall.y"        "distanceFromTarget"       
[47] "distance_from_qb"          "snap_to_throw"            
[49] "var_xs"                    "var_ys"                   
[51] "var_o"                     "avg_speed"                
[53] "var_speed"                 "look_at_qb"               
[55] "avg_dist"                  "var_dist"                 
[57] "avg_dir_diff"              "avg_speed_diff"           
[59] "pct_first_off"             "pct_last_off"             
[61] "last_off"                  "first_off"                
[63] "var_look_at_qb"            "dist_los"                 
[65] "complete"                  "covering_target"          
[67] "covering_first_off"

In [15]:
write.csv(rand_forest, 'rand_forest.csv')

In [16]:
colnames(full_data)

[1] "gameId"                    "playId"                   
 [3] "frameId"                   "off_nflId"                
 [5] "nflId"                     "displayName"              
 [7] "dist_from_off"             "nearest_is_target"        
 [9] "distanceFromBall.x"        "off_distanceFromBall"     
[11] "ball_behind_db"            "diff_in_speed"            
[13] "diff_in_acceleration"      "diff_in_dir"              
[15] "receiver_behind_db"        "receiver_distance_from_qb"
[17] "num_of_receivers_nearest"  "dist_rate_of_change"      
[19] "his_off_most_open"         "passResult"               
[21] "targetNflId"               "x"                        
[23] "y"                         "o"                        
[25] "dir"                       "s"                        
[27] "a"                         "event"                    
[29] "epa"                       "position"                 
[31] "route"                     "isDefensivePI"            
[33] "targetedReceiver"          "team_name"                
[35] "ball_x"                    "ball_y"                   
[37] "ball_snap_x"               "max_space_available"      
[39] "target_x"                  "target_y"                 
[41] "qb_x"                      "qb_y"                     
[43] "qb_s"                      "qb_a"                     
[45] "distanceFromBall.y"        "distanceFromTarget"       
[47] "distance_from_qb"          "snap_to_throw"            
[49] "var_xs"                    "var_ys"                   
[51] "var_o"                     "avg_speed"                
[53] "var_speed"                 "look_at_qb"               
[55] "avg_dist"                  "var_dist"                 
[57] "avg_dir_diff"              "avg_speed_diff"           
[59] "pct_first_off"             "pct_last_off"             
[61] "last_off"                  "first_off"                
[63] "var_look_at_qb"            "dist_los"                 
[65] "complete"                  "covering_target"          
[67] "covering_first_off"

In [17]:
pred_data <- full_data %>%

select(dist_from_off, dist_los,
       qb_s, qb_a, max_space_available, distance_from_qb, receiver_distance_from_qb,
       ball_behind_db, diff_in_speed, diff_in_acceleration, diff_in_dir,
       his_off_most_open, receiver_behind_db, dist_rate_of_change,
       gameId, playId, frameId, off_nflId, nflId, displayName, event)

In [18]:
write.csv(pred_data, 'rf_pred.csv')